In [1]:
from src.preprocessing.preprocess_HealthyArchive import OXPreprocessor

OXPreprocessor().process()

/Users/Hanita/Library/Python/3.12/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Processing Oxford cells: 100%|██████████| 8/8 [00:01<00:00,  5.41it/s]


(8, 0)

In [3]:
from src.preprocessing.battery_data import BatteryData
oxford_data = BatteryData.load('data/preprocessed/oxford/OX_1-1_pouch_LCO_40C_0-100_2-1.84C_a_timeseries.pkl')
oxford_data.print_description()

**************description of battery cell OX_1-1_pouch_LCO_40C_0-100_2-1.84C_a_timeseries**************
cell_id: OX_1-1_pouch_LCO_40C_0-100_2-1.84C_a_timeseries
organization: oxford
timeseries_data: [<src.preprocessing.battery_data.TimeseriesData object at 0x121579ac0>]
is_healthy: True
battery_type: LCO
anode_material: graphite
cathode_material: LCO
nominal_capacity_in_Ah: 0.72
form_factor: pouch


In [6]:
ts = next(t for t in oxford_data.timeseries_data)
ts.display()

,Time,Temp (°C)
0,0.000000e+00,40.026
1,9.990000e-01,40.051
2,1.999000e+00,39.988
3,2.999000e+00,39.976
4,3.999000e+00,39.964
...,...,...
460376,5.518128e+07,40.737
460377,5.518128e+07,40.750
460378,5.518128e+07,40.688
460379,5.518128e+07,40.725
